# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [104]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [105]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [106]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)
html = browser.html

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [107]:
# Create a Beautiful Soup Object
soup1 = soup(html, 'html.parser')
soup1

<html lang="en"><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="" name="description"/>
<title>Mars Temperature Data</title>
<link href="css/bootstrap.min.5.2.2.css" rel="stylesheet" type="text/css"/>
<link href="css/temp.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<main>
<div class="container py-4">
<header class="pb-3 mb-4 border-bottom">
<a class="d-flex align-items-center text-dark text-decoration-none" href="/">
<span class="fs-4">Mission To Mars</span>
</a>
</header>
<div class="p-5 mb-4 bg-light rounded-3">
<div class="container-fluid py-5">
<h1 class="display-5 fw-bold">Mars Temperature Data</h1>
<p class="col-md-8 fs-4">This is Mars data.</p>
<table class="table">
<tbody><tr>
<th>id</th>
<th>terrestrial_date</th>
<th>sol</th>
<th>ls</th>
<th>month</th>
<th>min_temp</th>
<th>pressure</th>
</tr>
<tr class="data-row">
<td>2</td>
<td>2012-08-16</td>
<td>10</td>
<td>155</td>
<td>6</td>
<td>-75.0</td>
<td>

In [108]:
# Extract all rows of data
# Note that "data" is not meant to include hearder 
header = soup1.find_all("th")
header 
data_rows = soup1.find_all("tr", class_='data-row')
data_rows


[<tr class="data-row">
 <td>2</td>
 <td>2012-08-16</td>
 <td>10</td>
 <td>155</td>
 <td>6</td>
 <td>-75.0</td>
 <td>739.0</td>
 </tr>,
 <tr class="data-row">
 <td>13</td>
 <td>2012-08-17</td>
 <td>11</td>
 <td>156</td>
 <td>6</td>
 <td>-76.0</td>
 <td>740.0</td>
 </tr>,
 <tr class="data-row">
 <td>24</td>
 <td>2012-08-18</td>
 <td>12</td>
 <td>156</td>
 <td>6</td>
 <td>-76.0</td>
 <td>741.0</td>
 </tr>,
 <tr class="data-row">
 <td>35</td>
 <td>2012-08-19</td>
 <td>13</td>
 <td>157</td>
 <td>6</td>
 <td>-74.0</td>
 <td>732.0</td>
 </tr>,
 <tr class="data-row">
 <td>46</td>
 <td>2012-08-20</td>
 <td>14</td>
 <td>157</td>
 <td>6</td>
 <td>-74.0</td>
 <td>740.0</td>
 </tr>,
 <tr class="data-row">
 <td>57</td>
 <td>2012-08-21</td>
 <td>15</td>
 <td>158</td>
 <td>6</td>
 <td>-78.0</td>
 <td>740.0</td>
 </tr>,
 <tr class="data-row">
 <td>68</td>
 <td>2012-08-22</td>
 <td>16</td>
 <td>158</td>
 <td>6</td>
 <td>-77.0</td>
 <td>740.0</td>
 </tr>,
 <tr class="data-row">
 <td>79</td>
 <td>2012-08-

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [236]:
from pprint import pprint

rows_list = []
temperature_data_dictionary_list = []
dict_list = []
columns = []
df=[]
df1=[]
count = 0
header_list = ['id','terrestrial_date','sol','ls','month','min_temp','pressure']

print("(1) header list = ", header_list)
data_list = [id, terrestrial_date, sol, ls, month, min_temp, pressure]
df = pd.DataFrame(columns = header_list)
df1 = pd.DataFrame()
# df.columns = ['id','terrestrial_date','sol','ls','month','min_temp','pressure']
print("(2) df = ", df) 
# # headers = soup1.find_all("th")
# # headers
# for col in headers:

# Create an empty list


# Loop through the scraped data to create a list of rows
for row in soup1.find_all('tr', class_= 'data-row'): #.tbody.find("tr", "data-row"):    
    # Find all data for each column
    columns = row.find_all('td')
#    column_data = [column.get_text(strip=True) for column in columns]
    id = columns[0].text.strip()
    terrestrial_date = columns[1].text.strip()
    sol = columns[2].text.strip()
    ls = columns[3].text.strip()
    month = columns[4].text.strip()
    min_temp = columns[5].text.strip()
    pressure = columns[6].text.strip()
#     df = df.append({'id': stripped_columns[0],  
    
    if(columns != []):
        count = count + 1
        print("(4) df1 = ",df1, "count = ",  count)
        df1 = {'id': id,  
                         'terrestrial_date': terrestrial_date, 
                         'sol': sol, 
                         'ls': ls, 
                         'month': month, 
                         'min_temp': min_temp, 
                         'pressure':pressure}
        df.loc[len(df)] = df1

print
print
print
df
        #sdf
        # ignore_index=True)
        # print("(4)", id, terrestrial_date, sol, ls, month, min_temp, pressure)
        # #print(stripped_columns[0])
    
        # data_list = [id, terrestrial_date, sol, ls, month, min_temp, pressure]
        # df1 = pd.DataFrame(columns = column_data)
        # count = count + 1
        # print("df1 = ",df1, "count = ",  count)
        
        # df_concatenated = pd.concat([df, df1], ignore_index=True)
        # print(df_concatenated)



(1) header list =  ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
(2) df =  Empty DataFrame
Columns: [id, terrestrial_date, sol, ls, month, min_temp, pressure]
Index: []
(4) df1 =  Empty DataFrame
Columns: []
Index: [] count =  1
(4) df1 =  {'id': '2', 'terrestrial_date': '2012-08-16', 'sol': '10', 'ls': '155', 'month': '6', 'min_temp': '-75.0', 'pressure': '739.0'} count =  2
(4) df1 =  {'id': '13', 'terrestrial_date': '2012-08-17', 'sol': '11', 'ls': '156', 'month': '6', 'min_temp': '-76.0', 'pressure': '740.0'} count =  3
(4) df1 =  {'id': '24', 'terrestrial_date': '2012-08-18', 'sol': '12', 'ls': '156', 'month': '6', 'min_temp': '-76.0', 'pressure': '741.0'} count =  4
(4) df1 =  {'id': '35', 'terrestrial_date': '2012-08-19', 'sol': '13', 'ls': '157', 'month': '6', 'min_temp': '-74.0', 'pressure': '732.0'} count =  5
(4) df1 =  {'id': '46', 'terrestrial_date': '2012-08-20', 'sol': '14', 'ls': '157', 'month': '6', 'min_temp': '-74.0', 'pressure': '740.0'} c

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0
...,...,...,...,...,...,...,...
1862,1889,2018-02-23,1973,133,5,-78.0,730.0
1863,1892,2018-02-24,1974,134,5,-77.0,729.0
1864,1894,2018-02-25,1975,134,5,-76.0,729.0
1865,1893,2018-02-26,1976,135,5,-77.0,728.0


In [186]:
from pprint import pprint
header_list = ['id','terrestrial_date','sol','ls','month','min_temp','pressure']
df = pd.DataFrame(columns = header_list)
print("(1)  header list = ", header_list)
# df.columns = ['id','terrestrial_date','sol','ls','month','min_temp','pressure']
print("(2)  df = ", df) 
# # headers = soup1.find_all("th")
# # headers
# for col in headers:

# Create an empty list

rows_list = []
temperature_data_dictionary_list = []
dict_list = []
columns = []
df1=[]
count = 0
# Loop through the scraped data to create a list of rows
for row in data_rows: #.tbody.find("tr", "data-row"):    
    # Find all data for each column
    columns = row.find_all('td')
    stripped_columns = [column.get_text(strip=True) for column in columns]
    print("(3)  ", id, terrestrial_date, sol, ls, month, min_temp, pressure)
    #print(stripped_columns[0])

    data_list = [id, terrestrial_date, sol, ls, month, min_temp, pressure]
    df1 = pd.DataFrame(columns = data_list)
    print("df1 = ",df1, "count = ",  count + 1)
    df_concatenated = pd.concat([df, df1], ignore_index=True)
    print('df_concatenated = ', df_concatenated)

    # new_row = [{'id': id,
    #             'terrestrial_date': terrestrial_date,
    #             'sol': sol,
    #             'ls': ls,
    #             'month': month,
    #             'min_temp': min_temp,
    #             'pressure':pressure}]

    # #print(stripped_columns[6])
    #df = df.concat([df, pd.DataFrame([new_row])], ignore_index=True)

    
    
#     if(columns != []):
#         id = columns[0].text.strip()
#         terrestrial_date = columns[1].text.strip()
#         sol = columns[2].text.strip()
#         ls = columns[3].text.strip()
#         month = columns[4].text.strip()
#         min_temp = columns[5].text.strip()
#         pressure = columns[6].text.strip()
#         df = df.append({'id': stripped_columns[0],  
#                         'terrestrial_date': stripped_columns[1], 
#                         'sol': sol, 
#                         'ls': ls, 
#                         'month': month, 
#                         'min_temp': min_temp, 
#                         'pressure':pressure}, 
#                        ignore_index=True)

# df
# for row in all_rows:
#     row_data = row.find_all('tr', class_="data-row")  #.text.strip()
#     rows_list.append(row_data)

rows_list
                    


(1)  header list =  ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
(2)  df =  Empty DataFrame
Columns: [id, terrestrial_date, sol, ls, month, min_temp, pressure]
Index: []
(3)   2 2012-08-16 10 155 6 -75.0 739.0
df1 =  Empty DataFrame
Columns: [2, 2012-08-16, 10, 155, 6, -75.0, 739.0]
Index: [] count =  1
df_concatenated =  Empty DataFrame
Columns: [id, terrestrial_date, sol, ls, month, min_temp, pressure, 2, 2012-08-16, 10, 155, 6, -75.0, 739.0]
Index: []
(3)   2 2012-08-16 10 155 6 -75.0 739.0
df1 =  Empty DataFrame
Columns: [2, 2012-08-16, 10, 155, 6, -75.0, 739.0]
Index: [] count =  1
df_concatenated =  Empty DataFrame
Columns: [id, terrestrial_date, sol, ls, month, min_temp, pressure, 2, 2012-08-16, 10, 155, 6, -75.0, 739.0]
Index: []
(3)   2 2012-08-16 10 155 6 -75.0 739.0
df1 =  Empty DataFrame
Columns: [2, 2012-08-16, 10, 155, 6, -75.0, 739.0]
Index: [] count =  1
df_concatenated =  Empty DataFrame
Columns: [id, terrestrial_date, sol, ls, month, min_te

[]

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.DataFrame(rows_list,columns=header_list)





# df = pd.DataFrame(columns=['id','terrestrial_date','sol','ls','month','min_temp','min_temp'])
# for row in rows:
#     df.append({'id': 'Value1', 
#                'terrestrial_date': 'Value2',
#                'sol': 'Value2',
#                'ls': 'Value2',
#                'month': 'Value2',
#                'min_temp': 'Value2',
#                'min_temp': 'Value2'})
               

#df

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
df = pd.read_html(url)
df

In [ ]:
# Confirm DataFrame was created successfully


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column


In [ ]:
# Change data types for data analysis


In [ ]:
# Confirm type changes were successful by examining data types again


### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?


In [ ]:
# 2. How many Martian days' worth of data are there?


In [ ]:
# 3. What is the average low temperature by month?


In [ ]:
# Plot the average temperature by month


In [ ]:
# Identify the coldest and hottest months in Curiosity's location


In [ ]:
# 4. Average pressure by Martian month


In [ ]:
# Plot the average pressure by month


In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV


In [ ]:
browser.quit()